In [80]:
import pandas as pd
import pyarrow as pa
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import ranksums, ttest_ind
import umap
from sklearn.preprocessing import StandardScaler
import numpy as np

In [81]:
data = Path("./").resolve().parent / "data"
concat_breast = data / "concat_breast_df_patient.pkl"
concat_lung= data / "concat_lung_df_patient.pkl"
til_breast = data / "breast_filtered_tils_patient.pkl"
til_lung = data / "lung_filtered_tils_patient.pkl"
if concat_breast.exists():
    bdf = pd.read_pickle(concat_breast).reset_index()
if concat_lung.exists():
    ldf = pd.read_pickle(concat_lung).reset_index()
if til_breast.exists():
    filt_bdf = pd.read_csv(til_breast)
if til_lung.exists():
    filt_ldf = pd.read_csv(til_lung)

In [ ]:
def print_value_counts(df, index_name, df_type, percentage=False):
    vc = df[index_name].value_counts()
    if percentage:
        vc = (vc / len(df) * 100).round(1)
        print(f"This is the percentage for {df_type}")
    else:
        print(f"This is the value counts for {df_type}\ntotal length: {len(df)}")

    headers = vc.index.tolist()
    values = vc.tolist()
    col_widths = int(max([max(len(str(h)), len(str(v))) for h, v in zip(headers, values)]))
    vc = vc.sort_index()
    he = [f"{h:{col_widths}}" for h in headers] 
    va = [f"{v:{col_widths}}" for v in values]
    border = '_'*col_widths*(len(headers)+2)
    print(f"{border}\n{"\t".join(he)}\n{"\t".join(va)}\n\n")

print_value_counts(filt_ldf, "level_0", "Filtered Lung")
print_value_counts(filt_ldf, "level_0", "Filtered Lung", percentage=True)
print_value_counts(filt_bdf, "level_0", "Filtered Breast")
print_value_counts(filt_bdf, "level_0", "Filtered Breast", percentage=True)
print_value_counts(ldf, "level_0", "Lung")
print_value_counts(ldf, "level_0", "Lung", percentage=True)
print_value_counts(bdf, "level_0", "Breast")
print_value_counts(bdf, "level_0", "Breast", percentage=True)

In [ ]:
def col_df(colname: str,bdf: pd.DataFrame, ldf: pd.DataFrame) -> pd.DataFrame:
    tmpb = bdf[colname].reset_index(drop=True).rename("Breast")
    tmpl = ldf[colname].reset_index(drop=True).rename("Lung")
    return pd.concat([tmpb, tmpl],axis=1)

def convert_pvalue_to_asterisks(pvalue):
    if pvalue <= 0.0001:
        return "****"
    elif pvalue <= 0.001:
        return "***"
    elif pvalue <= 0.01:
        return "**"
    elif pvalue <= 0.05:
        return "*"
    return "ns"

In [1]:
def bxplt(input_ser: pd.Series, title: str):
    ttest_res = ttest_ind(input_ser["Breast"].dropna(), input_ser["Lung"].dropna())
    print(ttest_res.pvalue)
    plt.figure()
    sns.boxplot(data=input_ser, showfliers=False, palette="Set3")
    if ttest_res.pvalue < 0.05:
        plt.text(0.9,0.95,convert_pvalue_to_asterisks(ttest_res.pvalue), transform=plt.gca().transAxes)
    plt.xlabel("All Cells")
    plt.title(title)
    plt.show()
    
nucl_area = col_df("Nucleus: Area", bdf,ldf)
nucl_area_til = col_df("Nucleus: Area", filt_bdf,filt_ldf)
bxplt(nucl_area, "Nucleus Area all cells")
bxplt(nucl_area_til, "Nucleus Area TILs")

NameError: name 'pd' is not defined

In [ ]:
plt.figure()
plt.scatter(bdf["Nucleus: Area"], bdf["Nucleus: Hematoxylin OD mean"], color="grey",alpha=0.4,s=0.8)
plt.scatter(filt_bdf["Nucleus: Area"], filt_bdf["Nucleus: Hematoxylin OD mean"], color="blue",alpha=0.4,s=0.9)
plt.ylabel("Nucleus: Hematoxylin OD mean")
plt.xlabel("Nucleus: Area")
plt.legend(["All breast cells", "Breast TILs"], loc="upper right")
plt.show()

In [ ]:
plt.figure()
plt.scatter(ldf["Nucleus: Area"], ldf["Nucleus: Hematoxylin OD mean"],color="grey",alpha=0.4,s=0.9)
plt.scatter(filt_ldf["Nucleus: Area"], filt_ldf["Nucleus: Hematoxylin OD mean"], color="red",alpha=0.4,s=0.9)
plt.ylabel("Nucleus: Hematoxylin OD mean")
plt.xlabel("Nucleus: Area")
plt.legend(["All lung cells", "Lung TILs"])
plt.show()

In [ ]:
filt_bdf["Tissue"] = "Breast"
filt_ldf["Tissue"] = "Lung"
concat_tils = pd.concat([filt_bdf, filt_ldf], axis=0)
cols_list = list(concat_tils.columns)

In [ ]:
filt_bdf.head()

In [ ]:
#concat_tils[cols_list[7:-1]].head()
ctils = concat_tils[cols_list[7:-1]].reset_index().drop("level_1", axis=1).rename(columns={"level_0": "Patient"})
ctils_shape = ctils[["Patient"] + cols_list[7:15]]
ctils_hema = ctils[["Patient"] + cols_list[15:21]]
ctils_eosin = ctils[["Patient"] + cols_list[21:27]]

In [ ]:
def umap_embedding(input_df: pd.DataFrame) -> pd.DataFrame:
    patient_types = set(input_df.Patient)
    scaled_tils= StandardScaler().fit_transform(input_df.drop("Patient", axis=1))
    reducer = umap.UMAP()
    embedding = reducer.fit_transform(scaled_tils)
    breast_embeds = [(x , embedding[input_df.Patient == x]) for x in patient_types if "BRCA" in x]
    lung_embeds = [(x , embedding[input_df.Patient == x]) for x in patient_types if "LUAD" in x]
    return breast_embeds, lung_embeds

ctils_shape_drop1 = ctils_shape[ctils_shape.Patient != "LUAD-1"]
breast_embed_sdrop1, lung_embed_sdrop1 = umap_embedding(ctils_shape_drop1)
breast_embed_shape, lung_embed_shape = umap_embedding(ctils_shape)
breast_embed_hema, lung_embed_hema = umap_embedding(ctils_hema)
breast_embed_eosin, lung_embed_eosin = umap_embedding(ctils_eosin)


In [ ]:
breast_embed, lung_embed = umap_embedding(ctils)
all_breast_embed, all_lung_embed = umap_embedding()

In [ ]:
def plot_scatter(breast_embeds, lung_embeds, title_str,lung_only=False, breast_only=False):
    blue4 = sns.color_palette("Blues", 5)
    red4 = sns.color_palette("Reds", 5)
    hls = sns.color_palette("hls", 10)
    plt.figure()
    if not lung_only:
        [ sns.scatterplot(x=x[1][:,0], y=x[1][:,1], alpha=0.9,color = blue4[i], label=x[0]) for i,x in enumerate(breast_embeds) ]
    if not breast_only:
        [ sns.scatterplot(x=x[1][:,0], y=x[1][:,1], alpha=0.9,color = red4[i], label=x[0]) for i,x in enumerate(lung_embeds) ]
    plt.gca().set_aspect('equal', 'datalim')
    plt.legend()
    plt.title(title_str)
    plt.show()
    plt.close()

plot_scatter(breast_embed_shape, lung_embed_shape, "UMAP projection of TILs based only on Shape Features")
plot_scatter(breast_embed_shape, lung_embed_shape, "UMAP projection of Lung TILs based only on Shape Features", lung_only=True)
plot_scatter(breast_embed_shape, lung_embed_shape, "UMAP projection of Breast TILs based only on Shape Features", breast_only=True)

In [ ]:
plot_scatter(breast_embed_hema, lung_embed_hema, "UMAP projection of TILs based only on Hematoxylin Features")
plot_scatter(breast_embed_eosin, lung_embed_eosin, "UMAP projection of TILs based only on Eosin Features")
plot_scatter(breast_embed_sdrop1, lung_embed_sdrop1, "UMAP projection of TILs based only on Shape Features without LUAD-1")

In [ ]:
plot_scatter(breast_embed, lung_embed, "UMAP projection of TILs based on all features")
plot_scatter(all_breast_embed, all_lung_embed, "UMAP projection of all cells based on all features")

In [ ]:
ctils.head()

In [ ]:
from sklearn.svm import OneClassSVM
def onesvm(input_df):
    scaled_tils= StandardScaler().fit_transform(input_df.drop("Patient", axis=1))
    model = OneClassSVM(nu=0.1, kernel='rbf', gamma='scale')
    model.fit(scaled_tils)
    outliers = model.predict(scaled_tils)
    return outliers
breast_total = ctils[ctils.Patient.str.contains("BRCA")]
lung_total = ctils[ctils.Patient.str.contains("LUAD")]
breast_outliers = onesvm(breast_total)
lung_outliers = onesvm(lung_total)

In [ ]:
bt_svm = breast_total.Patient[breast_outliers == -1].value_counts()
lt_svm = lung_total.Patient[lung_outliers == -1].value_counts()
#print(bt_svm)
#print(bt_svm / len(breast_total) * 100)
#print(f"Percentage of outliers {len(np.where(breast_outliers == -1)[0]) / len(breast_outliers) * 100}")
print(lt_svm)
print(lt_svm / len(lung_total) * 100)
print(f"Percentage of outliers {len(np.where(lung_outliers == -1)[0]) / len(lung_outliers) * 100}")

In [ ]:
breast_total.shape

In [ ]:

ranksums(filt_bdf["Nucleus: Area"], filt_ldf["Nucleus: Area"])